In [3]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('../data/processed/processed_customer_data_v1.csv')

scaler = MinMaxScaler()

In [4]:
# Define reference_date
reference_date = pd.to_datetime('2024-05-01')
print("reference_date:\n", reference_date.date())

df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')
df['last_active_date'] = pd.to_datetime(df['last_active_date'], errors='coerce')

# Recency: Days since last login or product usage
df['recency'] = (reference_date - df['last_active_date']).dt.days
print("Recency:\n", df['recency'].head(5))

# Frequency: Login count, feature usage count
df['frequency'] = (df['logins_last_30_days'] * 0.7) + (df['feature_usage_score'] * 10 * 0.3)
print("Frequency:\n", df['frequency'].head(10))

# Monetary: Monthly spend or LTV (months x monthly plan)
# df['months_active'] = (reference_date.year - pd.to_datetime(df['signup_date']).dt.year) * 12 + (reference_date.month - df['signup_date'].dt.month)
df['months_active'] = (reference_date.year - df['signup_date'].dt.year) * 12 + (reference_date.month - df['signup_date'].dt.month)
print("Months active:\n", df['months_active'].head(10))

df['ltv'] = df['monthly_spend'] * df['months_active']
print("LTV:\n", df['ltv'].head(10))

# Support burden: Tickets raised
# already exists

# Engagement score: Composite of usage, tickets and recency
# Normalize frequency (higher is better)
df['frequency_scaled'] = scaler.fit_transform(df[['frequency']])

# Normalize recency (lower is better, so invert the scale)
df['recency_scaled'] = 1 - scaler.fit_transform(df[['recency']])

# Normalize support tickets (lower is better, so invert the scale)
df['support_scaled'] = 1 - scaler.fit_transform(df[['support_tickets']])

df['engagement_score'] = (
    0.4 * df['frequency_scaled'] +    # Weight for frequency (usage)
    0.3 * df['recency_scaled'] +      # Weight for recency (activity)
    0.2 * df['support_scaled']        # Weight for support tickets (less is better)
)
print("Engagement score:\n", df['engagement_score'].head(10))

reference_date:
 2024-05-01
Recency:
 0    21
1     4
2     1
3    24
4     9
Name: recency, dtype: int64
Frequency:
 0     9.18
1     6.29
2    10.63
3     5.59
4    11.22
5     8.45
6     6.54
7     8.45
8     5.29
9     7.80
Name: frequency, dtype: float64
Months active:
 0    27
1    20
2    35
3    29
4    21
5    34
6     8
7    30
8    14
9    25
Name: months_active, dtype: int32
LTV:
 0      486.27
1     9373.60
2     1820.70
3     1593.84
4      419.58
5      651.78
6      176.80
7    50004.60
8      972.02
9     1265.25
Name: ltv, dtype: float64
Engagement score:
 0    0.684686
1    0.583630
2    0.623208
3    0.562173
4    0.694421
5    0.651671
6    0.608212
7    0.672500
8    0.555347
9    0.617987
Name: engagement_score, dtype: float64
